# Dependencies

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from copy import deepcopy
import seaborn as sns
from imblearn.over_sampling import RandomOverSampler
import yaml
import os
import sys

sys.path.append(os.path.join(os.getcwd(), 'src'))
import pickle
import joblib
import src.utils as utils

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from copy import deepcopy
import seaborn as sns
from imblearn.over_sampling import RandomOverSampler
import yaml
import pickle
import joblib
import os
import src.utils as utils
import importlib
importlib.reload(utils)

## load configuration file
with open("config.yaml", "r") as f:
    config = yaml.safe_load(f)
root_path = os.getcwd()

In [4]:
## load constant variables
TARGET_COL = config['data_source']['target_col']
TEST_SZ = config['data_source']['test_size']
SEED_ST = config['data_source']['random_state']
NUM_COLS = config['data_source']['num_cols']
CAT_COLS = config['data_source']['cat_cols']
CORR_COLS = config['data_source']['corr_cols']

## data
X_TRAIN_PATH = config['processed_files']['X_train']
X_TEST_PATH = config['processed_files']['X_test']
X_VALID_PATH = config['processed_files']['X_valid']
y_TRAIN_PATH = config['processed_files']['y_train']
y_TEST_PATH = config['processed_files']['y_test']
y_VALID_PATH = config['processed_files']['y_valid']
X_TRAIN_ROS_PATH = config['ROS_file']['X_train']
y_TRAIN_ROS_PATH = config['ROS_file']['y_train']

## Desirialize Pickle Data
var_name = ["X_train_prep", "X_test_prep", "X_valid_prep", "y_train_prep", "y_test_prep", "y_valid_prep", "X_train_ros", "y_train_ros"]
stock_list = [X_TRAIN_PATH, X_TEST_PATH, X_VALID_PATH, y_TRAIN_PATH, y_TEST_PATH, y_VALID_PATH, X_TRAIN_ROS_PATH, y_TRAIN_ROS_PATH]
for var_name, path in zip(var_name, stock_list):
    globals()[var_name] = utils.deserialize_data(path)

In [5]:
X_train_ros.isna().sum()

person_age                       0
person_income                    0
person_emp_length                0
loan_amnt                        0
loan_int_rate                    0
loan_percent_income              0
cb_person_cred_hist_length       0
home_ownership_MORTGAGE          0
home_ownership_OTHER             0
home_ownership_OWN               0
home_ownership_RENT              0
loan_intent_DEBTCONSOLIDATION    0
loan_intent_EDUCATION            0
loan_intent_HOMEIMPROVEMENT      0
loan_intent_MEDICAL              0
loan_intent_PERSONAL             0
loan_intent_VENTURE              0
loan_grade_A                     0
loan_grade_B                     0
loan_grade_C                     0
loan_grade_D                     0
loan_grade_E                     0
loan_grade_F                     0
loan_grade_G                     0
default_onfile_N                 0
default_onfile_Y                 0
dtype: int64

In [6]:
y_train_ros.isna().sum()

0

In [10]:
X_train_ros

,person_age,person_income,person_emp_length,loan_amnt,loan_int_rate,loan_percent_income,cb_person_cred_hist_length,home_ownership_MORTGAGE,home_ownership_OTHER,home_ownership_OWN,...,loan_intent_VENTURE,loan_grade_A,loan_grade_B,loan_grade_C,loan_grade_D,loan_grade_E,loan_grade_F,loan_grade_G,default_onfile_N,default_onfile_Y
0,45,37500,1.0,5000,11.49,0.13,16,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,25,50000,5.0,12000,7.88,0.24,2,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,24,78000,4.0,30000,10.99,0.38,4,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
3,31,78504,2.0,10000,11.41,0.13,7,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,26,14000,2.0,4000,10.99,0.29,3,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40555,22,53000,5.0,25000,13.43,0.47,4,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
40556,25,57504,2.0,16500,9.32,0.29,3,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
40557,28,60675,6.0,14000,15.62,0.23,10,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
40558,33,32400,8.0,11000,6.99,0.34,10,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [15]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [53]:
## import metrics dependencies

from sklearn.model_selection import cross_val_predict, GridSearchCV
from sklearn.metrics import balanced_accuracy_score, roc_auc_score, roc_curve, auc
from sklearn.metrics import precision_score, recall_score, classification_report
from sklearn.metrics import confusion_matrix, make_scorer
from sklearn.metrics import precision_recall_curve
from copy import deepcopy
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier

Based on the previous exercises, there are 3 algorithms that performed well on this data set: XG-Boost Classifier, Logistic Regression and Random Forest. In this instance we will perform hyperparameter search for each algorithm with cross validated grid search.

## XG Boost

In [44]:
def gridsearch_train(classifier, hyperparams: dict, CPU_unit, train_input:pd.DataFrame, train_output:pd.DataFrame):
    estimator = GridSearchCV(
        estimator=classifier, param_grid=hyperparams, scoring='f1', verbose=3, n_jobs=CPU_unit, cv=3, error_score='raise')
    estimator.fit(train_input, train_output)

    optimal_params = estimator.best_params_
    print(f"\nBest params from grid search: {optimal_params}")
    print(f"With best params, f1 score is: {estimator.best_score_}")

    return optimal_params

In [45]:
####'#### XGBoost
clf_xgb = XGBClassifier(objective="binary:logistic", subsample=0.9, colsample_bytree=0.5, seed=SEED_ST)

# xgb_params_1 = {
#     "max_depth": [3, 4, 5],
#     "learning_rate": [0.1, 0.01, 0.05],
#     "gamma": [0, 0.25, 1],
#     "reg_lambda": [0, 1.0, 10.0],
#     "scale_pos_weight": [1, 3, 5]
# }
###################################  Results ####################################
# best params from grid search: {'gamma': 0, 'learning_rate': 0.1, 'max_depth': 3,
# 'reg_lambda': 0, 'scale_pos_weight': 1}

# more exploration should be done in learning rate and max depth
#################################################################################


# xgb_params_2 = {
#     "max_depth": [3, 5, 7],
#     "learning_rate": [0.1, 1, 10],
#     "gamma": [0],
#     "reg_lambda": [0],
#     "scale_pos_weight": [1]
# }

###################################  Results ####################################
# best params from grid search: {'gamma': 0, 'learning_rate': 0.1, 'max_depth': 3, 'reg_lambda': 0, 'scale_pos_weight': 1}

# result from the second hyperparameter show a stable learning rate. However we need to perform more search for max_depth
#################################################################################



xgb_params_3 = {
    "max_depth": [9, 12, 15],
    "learning_rate": [1],
    "gamma": [0],
    "reg_lambda": [0],
    "scale_pos_weight": [1]
}

###################################  Results ####################################
# best params from grid search: {'gamma': 0, 'learning_rate': 0.1, 'max_depth': 3, 'reg_lambda': 0, 'scale_pos_weight': 1}

# now all of our hyperparameters are stable. we can extract the result in a variable called xgb_opt_params to be used for our model training.
#################################################################################


xgb_opt_params = gridsearch_train(clf_xgb, hyperparams=xgb_params_3, CPU_unit=10, train_input=X_train_ros, train_output=y_train_ros)

Fitting 3 folds for each of 3 candidates, totalling 9 fits

Best params from grid search: {'gamma': 0, 'learning_rate': 1, 'max_depth': 12, 'reg_lambda': 0, 'scale_pos_weight': 1}
With best params, f1 score is: 0.9186187471265542


In [35]:
xgb_opt_params

{'gamma': 0,
 'learning_rate': 1,
 'max_depth': 12,
 'reg_lambda': 0,
 'scale_pos_weight': 1}

In [36]:
best_xgb = XGBClassifier(**xgb_opt_params)
best_xgb.fit(X_train_ros, y_train_ros)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=0, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=12, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

## Logistic Regression

In [48]:
######## Logistic Regression
clf_lgreg = LogisticRegression(random_state=SEED_ST)

# lgreg_params_1 = {
#     "penalty"           : ['l2', 'l1'],
#     "C"                 : [0.001, 0.01, 0.1],
#     "fit_intercept"     : [True, False],
#     "solver"            : ['liblinear', 'saga'],
#     "max_iter"          : [100, 250, 500],
# }

###################################  Results ####################################
# Best params from grid search: {'C': 0.1, 'fit_intercept': True, 'max_iter': 100, 'penalty': 'l1', 'solver': 'liblinear'}
# With best params, f1 score is: 0.7939004526536887

# we need to explore hyperparams of max_iter and C.
#################################################################################


lgreg_params_2 = {
    "penalty"           : ['l1'],
    "C"                 : [0.1, 1, 10],
    "fit_intercept"     : [True],
    "solver"            : ['liblinear'],
    "max_iter"          : [10, 50, 100],
}

###################################  Results ####################################
# Best params from grid search: {'C': 10, 'fit_intercept': True, 'max_iter': 10, 'penalty': 'l1', 'solver': 'liblinear'}
# With best params, f1 score is: 0.7947924346403775

# number of C is still not stable we can try more combination. there are warning to keep iteration higher than 10. therefore we will set the value to 100 as same as defualt value in sklearn.
#################################################################################

lgreg_params_3 = {
    "penalty"           : ['l1'],
    "C"                 : [10, 25, 50],
    "fit_intercept"     : [True],
    "solver"            : ['liblinear'],
    "max_iter"          : [100],
}


X_train_lgreg = np.ascontiguousarray(X_train_ros)
y_train_lgreg = np.ascontiguousarray(y_train_ros)

lgreg_opt_params = gridsearch_train(clf_lgreg, hyperparams=lgreg_params_3, CPU_unit=10, train_input=X_train_lgreg, train_output=y_train_lgreg)

Fitting 3 folds for each of 3 candidates, totalling 9 fits

Best params from grid search: {'C': 25, 'fit_intercept': True, 'max_iter': 100, 'penalty': 'l1', 'solver': 'liblinear'}
With best params, f1 score is: 0.7943684963923671


In [49]:
lgreg_opt_params

{'C': 25,
 'fit_intercept': True,
 'max_iter': 100,
 'penalty': 'l1',
 'solver': 'liblinear'}

In [51]:
best_lgreg = LogisticRegression(**lgreg_opt_params)
best_lgreg.fit(X_train_lgreg, y_train_lgreg)

LogisticRegression(C=25, penalty='l1', solver='liblinear')

## Dummy Classifier

In [ ]:
######## Random Forest
dum_params_1 = {
    "strategy": ["most_frequent", "uniform"],
    ""
}